а) Создать класс «Автомобиль» с полями «Марка», «Модель», «Год выпуска» и «Цвет». Создать хеш-таблицу для хранения объектов класса «Автомобиль» по ключу — номеру государственной регистрации.

In [6]:
import hashlib

class Car:
    def __init__(self, make, model, year, color):
        self.make = make
        self.model = model
        self.year = year
        self.color = color

class CarRegistry:
    def __init__(self, size):
        self.size = size
        self.table = [None] * size

    def hash_function(self, key): #использует хеш-функцию SHA-1 для преобразования ключа (составленного из марки, модели и года) в индекс в хеш-таблице.
        return int(hashlib.sha1(key.encode()).hexdigest(), 16) % self.size

    def insert(self, car): #вставляет автомобиль в хеш-таблицу, используя ключ и разрешая коллизии методом линейного пробирования.
        key = car.make + car.model + str(car.year)
        index = self.hash_function(key)
        while self.table[index] is not None:
            index = (index + 1) % self.size
        self.table[index] = car

    def find(self, key): #ищет автомобиль по ключу, используя тот же процесс хеширования и линейного пробирования для обхода коллизий.
        index = self.hash_function(key)
        while self.table[index] is not None:
            if self.table[index].make + self.table[index].model + str(self.table[index].year) == key:
                return self.table[index]
            index = (index + 1) % self.size
        return None

#Создаем CarRegistry размером 10
car_registry = CarRegistry(10)

#Создайте несколько объектов автомобилей.
car1 = Car("Toyota", "Camry", 2020, "Red")
car2 = Car("Honda", "Accord", 2021, "Blue")
car3 = Car("Ford", "Mustang", 2019, "Yellow")

#Вносим автомобили в реестр
car_registry.insert(car1)
car_registry.insert(car2)
car_registry.insert(car3)

#Найти машину по ключу
key = "ToyotaCamry2020"
found_car = car_registry.find(key)

print(found_car)


б) Написать функцию для нахождения наиболее часто встречающегося значения в хеш-таблице.

In [4]:
def find_most_frequent(hash_table):
    frequency = {} #для отслеживания частоты встречаемости элементов
    for item in hash_table: #В этой части кода мы увеличиваем счетчик для каждого элемента, встреченного в hash_table.
        if item is not None:
            if item not in frequency:
                frequency[item] = 0
            frequency[item] += 1

    most_frequent = None #Нахождение наиболее часто встречающегося элемента
    max_frequency = 0
    for item, frequency in frequency.items():
        if frequency > max_frequency:
            most_frequent = item
            max_frequency = frequency

    return most_frequent #Возвращение наиболее часто встречающегося элемента


в) Реализуйте хеш-таблицу для хранения информации о товарах на складе. Ключом является штрих-код товара, значение — объект, содержащий информацию о товаре (название, количество, цена и т.д.). Используйте метод разрешения коллизий методом открытой адресации с квадратичным пробированием и реализуйте возможность изменения количества товаров.

In [5]:
class Product:
    def __init__(self, name, quantity, price):
        self.name = name
        self.quantity = quantity
        self.price = price

class Warehouse: #инициализируется с заданным размером (size) и создает таблицу (список) для хранения объектов Product.
    def __init__(self, size):
        self.size = size
        self.table = [None] * size

    def hash_function(self, key): #использует хеш-функцию для вычисления индекса в таблице на основе имени продукта (key). Он возвращает вычисленный хеш по модулю размера таблицы.
        return int(hashlib.sha1(key.encode()).hexdigest(), 16) % self.size

    def insert(self, product): #добавляет продукт в хеш-таблицу. Он вычисляет индекс с помощью хеш-функции, а затем использует квадратичное пробирование (поиск свободной ячейки) для разрешения коллизий и вставки продукта
        key = product.name
        index = self.hash_function(key)
        offset = 1
        while self.table[index] is not None:
            index = (index + offset**2) % self.size
            offset += 1
        self.table[index] = product

    def find(self, key): # выполняет поиск продукта по имени (key) в хеш-таблице. Он также использует квадратичное пробирование для обхода таблицы и нахождения нужного продукта.

        index = self.hash_function(key)
        offset = 1
        while self.table[index] is not None:
            if self.table[index].name == key:
                return self.table[index]
            index = (index + offset**2) % self.size
            offset += 1
        return None

    def update_quantity(self, key, new_quantity): #обновляет количество продукта по заданному имени (key) на новое значение new_quantity, если продукт найден в таблице.
        product = self.find(key)
        if product is not None:
            product.quantity = new_quantity
            